### Imports

In [1]:
import os
import numpy as np
import random
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore') 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # hopefully nothing explodes

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
print(tf.version)
print(tf.config.list_physical_devices('GPU'))

<module 'tensorflow._api.v2.version' from '/home/zyzz/anaconda3/lib/python3.11/site-packages/tensorflow/_api/v2/version/__init__.py'>
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# For reproducible results
seed = 42
os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)  

## Data

In [3]:
# Constants
val_size = 0.2
data_path = 'training_dataset'
seq_length = 128     # predictions based on previous seq_length data entries
forecast_length = 9  # predicting forecast_length time steps into the future
sample_length = seq_length + forecast_length

In [4]:
# Read data (ignore categories)
training_data = np.load(os.path.join(data_path, 'training_data.npy'))
valid_periods = np.load(os.path.join(data_path, 'valid_periods.npy'))

# Filter out unvalid data
data = []
for i, row in enumerate(training_data):
    data.append(row[valid_periods[i][0]:valid_periods[i][1]])

print(f"({len(data)}, -)", valid_periods.shape)

(48000, -) (48000, 2)


In [5]:
# Convert time series to {x: sequences of length seq_length, y: values to be predicted from previous sequence}
def to_sequences(time_series):
    
    x = []
    y = []
    
    for i in range(time_series.shape[0]-seq_length-forecast_length+1):
        x.append(time_series[i:i+seq_length])
        y.append(time_series[i+seq_length:i+seq_length+forecast_length])  
    
    x = np.array(x)
    y = np.array(y)
    
    return {'x': x.reshape((x.shape[0], x.shape[1], -1)), 'y': y.reshape((y.shape[0], y.shape[1], -1))}

# Shuffle data
np.random.shuffle(data)
 
# Build sequences from the non-correlated time series, and append them to corresponding data set
# Note: there is no overlap between train and validation; each processed time series is used in train xor val
X_train, X_val = [], []
y_train, y_val = [], []
data = data
split_index = int((1-val_size)*len(data))
for i, time_series in enumerate(data): 
    if (len(time_series) >= sample_length): # assert we can draw at least one sample from the time_series
        sequences = to_sequences(time_series)
        if(i < split_index):
            X_train.append(sequences['x']) 
            y_train.append(sequences['y'])   
        else:
            X_val.append(sequences['x']) 
            y_val.append(sequences['y'])  

# Convert lists to nparrays 
X_train = np.concatenate(X_train, axis=0)
X_val = np.concatenate(X_val, axis=0)
y_train = np.concatenate(y_train, axis=0)
y_val = np.concatenate(y_val, axis=0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(3373049, 128, 1) (847359, 128, 1) (3373049, 9, 1) (847359, 9, 1)


## ML

In [6]:
input_shape = (X_train.shape[1], X_train.shape[2])
dropout_rate = 0.2
batch_size = 128
lstm_units = 128
epochs = 1000

def build_model(input_shape, lstm_units, dropout_rate):
    input_layer = tfkl.Input(shape=input_shape)
      
    # Block one; bidirectional LSTMs with regularizers, BNormalization and dropout
    x = tfkl.Bidirectional(
        tfkl.LSTM(units=lstm_units, return_sequences=True, kernel_regularizer=tfk.regularizers.l2(0.001)))(input_layer)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(dropout_rate)(x)

    # Block two; similar to previous but no bidirectionality or regularizer
    x = tfkl.LSTM(units=lstm_units // 2)(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(dropout_rate)(x)
    x = tfkl.Dense(units=forecast_length)(x)

    # Compile using MSE as loss function and the Adam optimizer
    model = tfk.Model(inputs=input_layer, outputs=x)
    model.compile(optimizer='adam', loss='mse')

    return model

In [7]:
# Stop training when validation loss stops improving, maintain best weights
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,         # how many epochs to check for improvement before stopping
    restore_best_weights=True,
)
    
# Build and train model
model = build_model(input_shape, lstm_units, dropout_rate)
history = model.fit(X_train,
                    y_train, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    validation_data=(X_val, y_val),
                    callbacks=early_stopping,
                    verbose=1)

# Evaluate on validation data
val_result = model.evaluate(X_val, y_val, verbose=0)
print(f"Val loss (MSE): {val_result}")

Epoch 1/1000


I0000 00:00:1702584325.096149    3522 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


26352/26352 [==============================] - 606s 23ms/step - loss: 0.0139 - val_loss: 0.0078
Epoch 2/1000
26352/26352 [==============================] - 585s 22ms/step - loss: 0.0072 - val_loss: 0.0077
Epoch 3/1000
26352/26352 [==============================] - 575s 22ms/step - loss: 0.0070 - val_loss: 0.0069
Epoch 4/1000
26352/26352 [==============================] - 580s 22ms/step - loss: 0.0069 - val_loss: 0.0067
Epoch 5/1000
26352/26352 [==============================] - 577s 22ms/step - loss: 0.0069 - val_loss: 0.0058
Epoch 6/1000
26352/26352 [==============================] - 577s 22ms/step - loss: 0.0068 - val_loss: 0.0058
Epoch 7/1000
26352/26352 [==============================] - 574s 22ms/step - loss: 0.0067 - val_loss: 0.0063
Epoch 8/1000
26352/26352 [==============================] - 570s 22ms/step - loss: 0.0067 - val_loss: 0.0056
Epoch 9/1000
26352/26352 [==============================] - 570s 22ms/step - loss: 0.0067 - val_loss: 0.0066
Epoch 10/1000
26352/26352 [=====

In [8]:
# Val loss for each prediction step
y_pred = model.predict(X_val)
for t in range(forecast_length):
    m = tf.keras.metrics.MeanSquaredError()
    m.update_state(y_val[:, t], y_pred[:, t])
    print(f'Val loss (MSE) {t+1}h forward: {m.result().numpy()}')

26480/26480 [==============================] - 180s 7ms/step
Val loss (MSE) 1h forward: 0.0026429244317114353
Val loss (MSE) 2h forward: 0.003449637209996581
Val loss (MSE) 3h forward: 0.004199344199150801
Val loss (MSE) 4h forward: 0.0049636890180408955
Val loss (MSE) 5h forward: 0.0057109203189611435
Val loss (MSE) 6h forward: 0.006271521095186472
Val loss (MSE) 7h forward: 0.006858121603727341
Val loss (MSE) 8h forward: 0.007437693886458874
Val loss (MSE) 9h forward: 0.007948368787765503


In [9]:
# save model
model.save('LSTM_v1')

INFO:tensorflow:Assets written to: LSTM_v1/assets


INFO:tensorflow:Assets written to: LSTM_v1/assets
